<a href="https://colab.research.google.com/github/justadudewhohacks/ipynbs/blob/master/age_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Dependencies


In [0]:
!apt-get install p7zip-full
!pip install -U -q PyDrive

### Download Data

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import os

if not os.path.exists('./data'):
  os.makedirs('./data')
if not os.path.exists('./data/utk'):
  os.makedirs('./data/utk')
if not os.path.exists('./data/appareal'):
  os.makedirs('./data/appareal')
if not os.path.exists('./data/wiki'):
  os.makedirs('./data/wiki')

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
  
print('downloading utk data...')
drive.CreateFile({ 'id': os.environ['utk_images_7z_id'] }).GetContentFile('./data/utk/images.7z')
drive.CreateFile({ 'id': os.environ['utk_landmarks_7z_id'] }).GetContentFile('./data/utk/landmarks.7z')

print('downloading appareal data...')
drive.CreateFile({ 'id': os.environ['appareal_labels_json_id'] }).GetContentFile('./data/appareal/labels.json')
drive.CreateFile({ 'id': os.environ['appareal_images_7z_id'] }).GetContentFile('./data/appareal/images.7z')
drive.CreateFile({ 'id': os.environ['appareal_landmarks_7z_id'] }).GetContentFile('./data/appareal/landmarks.7z')

print('downloading wiki data...')
drive.CreateFile({ 'id': os.environ['wiki_labels_json_id'] }).GetContentFile('./data/wiki/labels.json')
drive.CreateFile({ 'id': os.environ['wiki_images_7z_id'] }).GetContentFile('./data/wiki/images.7z')
drive.CreateFile({ 'id': os.environ['wiki_landmarks_7z_id'] }).GetContentFile('./data/wiki/landmarks.7z')
  
print('done!')

downloading utk data...
downloading appareal data...
downloading wiki data...
done!


### Unzip Data

In [0]:
!cd ./data/utk && p7zip -d ./images.7z
!cd ./data/utk && p7zip -d ./landmarks.7z


7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan ./            1 file, 216371463 bytes (207 MiB)

Extracting archive: ./images.7z
--
Path = ./images.7z
Type = 7z
Physical Size = 216371463
Headers Size = 309426
Method = LZMA2:24
Solid = +
Blocks = 1

  0%      1% 1        2% 380 - cropped-images/12_1_4_20170103200804119.jpg.chip_0.jpg                                                                   3% 417 - cropped-images/13_0_2_20170103201417686.jpg.chip_0.jpg                                                                 

In [0]:
!cd ./data/appareal && p7zip -d ./images.7z
!cd ./data/appareal && p7zip -d ./landmarks.7z


7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan ./            1 file, 80118409 bytes (77 MiB)

Extracting archive: ./images.7z
--
Path = ./images.7z
Type = 7z
Physical Size = 80118409
Headers Size = 73314
Method = LZMA2:24
Solid = +
Blocks = 1

  0%      3% 1        6% 369 - cropped-images/000472.jpg_face_0.jpg                                                 9% 370 - cropped-images/000473.jpg_face_0.jpg                                                11% 741 - cropped-images/000859.jpg_face_0.jpg                        

In [0]:
!cd ./data/wiki && p7zip -d ./images.7z
!cd ./data/wiki && p7zip -d ./landmarks.7z


7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan ./            1 file, 393939198 bytes (376 MiB)

Extracting archive: ./images.7z
--
Path = ./images.7z
Type = 7z
Physical Size = 393939198
Headers Size = 562067
Method = LZMA2:24
Solid = +
Blocks = 1

  0%      0% 1        1% 350 - cropped-images/01_1447301_1975-05-31_2009_0.jpg                                                            1% 351 - cropped-images/01_1451801_1894-03-19_1968_0.jpg                                                            2% 654 - cropped-images/